# Evolution of the overlap of two configurations given a network with a set of stored patterns 

This notebook studies the overlap as a funciton of time of two spin configurations that start off with a given difference, meaning the overlap is not 1. The network is defined as in "B. Derrida, E. Gardner, and A. Zippelius. An exactly solvable asymmetric neural network model. EPL, 4(2), 1987." and input is given to the network and scaled by an encoder. The encoder and each of the two configuraitons have a given overlap with the stored patterns in the network, which will affect the evolution of the overlap of the two configurations.

In [ ]:
#import packages

import sklearn as skl
import matplotlib.pyplot as plt
import numpy as np
import sys
import scipy.stats as sps
import scipy.sparse as ssp
from scipy.sparse import csr_matrix, coo_array
import scipy.integrate as integrate
import warnings
warnings.filterwarnings('ignore')

from sklearn.utils.validation import check_is_fitted
from sklearn.cluster import SpectralClustering
from scipy.io import loadmat

import networkx as nx

import math
import random

# 1. Model definition of network with memory scaffold

Network constructed according to the following reference: B. Derrida, E. Gardner, and A. Zippelius. An exactly solvable asymmetric neural network model. EPL, 4(2), 1987.

In [ ]:
class Ising_setup(skl.base.BaseEstimator, skl.base.TransformerMixin):
    np.random.seed(117)
    def __init__(self,n_spins = 1000, p=2, overlap = 0.8, overlap_encoder = 0.5, q = 0.8, n_C = 6):

        self.n_spins = n_spins #number of neurons in the network
        self.n_C = n_C #average graph connectivity
        self.p = p #number of patterns      
        self.overlap = overlap #quantify the amount of overal you want with the first pattern and the inital spins
        self.ov_encoder = overlap_encoder #quantify the overlap with the encoder
        self.q = q #overlap between the two spin configurations
    
    
    def getConnectivity(self):
        return self.n_C
    
    def getPatterns(self):
        return self.p    
    
    def randomPatterns(self):
        
        aligned_pattern = (np.random.randint(2,size = (self.n_spins, 1))*2 -1).reshape(1,self.n_spins)
             
        pattern_tensor = np.tile(aligned_pattern,(self.p,1))

        
        #Microscopic alignment patterns (scale with sqrt(N))  
        for i in range(self.p-1):
            idx_no_align_micro = random.sample(range(self.n_spins),
                                               int((1-((self.overlap)/np.sqrt(self.n_spins)))*self.n_spins))            
            pattern_tensor[i+1,idx_no_align_micro] = pattern_tensor[i+1,idx_no_align_micro]*-1
                    

        return pattern_tensor
    
    def alignConfig(self,pattern, indices):
        '''
        Function to generate a configuration or encoder with indices that are the same as a pattern. It is assumed
        that all patterns and configurations are of value +-1.
        '''
        config = pattern*-1
        config[indices] = pattern[indices]
        return config
    
    def generateSpins(self, pattern):
        '''
        Takes as an argument the pattern with the macroscopic overlap and generates two spin configuration with a given
        overlap among themselves (self.q) and the macroscopic pattern (self.overlap)
        '''
        #Make the two configurations overlap with the macroscopic pattern in self.overlap-(1-self.q)/2 fraction of 
        #indices (same ones). Make them differ at two different (1-self.q)/2 fraction of locations, where one 
        #configuration alignes and the other doesn't with the macroscopic pattern, and viceversa.
        
        #Flip both patterns and then align them
        spins1 = pattern*-1
        spins2 = pattern*-1
        
        half_diff = (1-self.q)/2     
        
        num_ov_same = int((self.overlap-half_diff)*self.n_spins)
        num_ov_diff = int(half_diff*self.n_spins)
        
        idx_align = random.sample(range(self.n_spins),num_ov_same +2*num_ov_diff)
        idx_ov_same = idx_align[0:num_ov_same]
        idx_ov_diff1 = idx_align[num_ov_same:num_ov_same+num_ov_diff]
        idx_ov_diff2 = idx_align[num_ov_same+num_ov_diff:num_ov_same+2*num_ov_diff]

        spins1[idx_ov_same] = pattern[idx_ov_same]
        spins2[idx_ov_same] = pattern[idx_ov_same]
        spins1[idx_ov_diff1] = pattern[idx_ov_diff1]
        spins2[idx_ov_diff2] = pattern[idx_ov_diff2]
        
        return spins1, spins2
        
        
    def generateEncoder(self, pattern):
        '''
        Takes the degree of overlap with the given pattern and generates an encoder accordingly
        '''
        idx_align = random.sample(range(self.n_spins),int(self.ov_encoder*self.n_spins))
        encoder = self.alignConfig(pattern, idx_align)
               
        return encoder       

    def setConnectivity(self):
        '''
        Set the connectivity matrix
        '''

        C_graph = nx.fast_gnp_random_graph(self.n_spins,self.n_C/self.n_spins,None,True)
        C_mat = nx.to_scipy_sparse_array(C_graph) #convert to sparse matrix
        C_mat.setdiag(0,k=0)#no self connectivity in the model
                
        return C_mat
    
    def setWeights(self):

        
        pattern_matrix = self.randomPatterns() #matrix with each of the patterns (n_spins, n_patterns)
        #interactions_across_patterns = pattern_matrix@pattern_matrix.T
        
        encoder = self.generateEncoder(pattern_matrix[0,:])
        spins1, spins2 = self.generateSpins(pattern_matrix[0,:])
        C_mat = self.setConnectivity()#connectivity matrix
        
        rows, cols = C_mat.nonzero()
        J_mat = C_mat.copy()
        
        for i,j in zip(rows,cols):
            J_mat[i,j] =pattern_matrix[:,i]@pattern_matrix[:,j]

        return J_mat, C_mat, pattern_matrix, spins1, spins2, encoder

# 2. Model updating

Class to define how the system evolves given a specific network definition

In [ ]:
import math
import random


class Ising_run(skl.base.BaseEstimator, skl.base.TransformerMixin):
    #np.random.seed(117)
    def __init__(self, spins1, spins2, p, J, encoder, r = 0.5, mean_u = 0, temperature = 0.01, input_u = True, 
                 input_same = True):
                
        self.temperature = temperature
        self.n_spins = spins1.shape[0]
        #pattern matrix, each row corresponds to a pattern, first row being the one with macroscopic overlap
        self.p = p #pattern matrix 
        
        #input parameters
        self.r = r
        self.mean_u = mean_u
        
        #initialize both configurations to be the same
        self.spins1 = spins1 #initialize spin configuration (column vector)
        self.spins2 = spins2 #initialize spin configuration (column vector)
        
        #define two fields
        self.J = J #weight matrix
        self.field_matrix1 = J@self.spins1
        self.field_matrix2 = J@self.spins2
        
        self.encoder = encoder
        
        self.input_u = input_u
        self.input_same = input_same

    def getPatterns(self):
        return self.p

    def getTemperature(self):
        return self.temperature

    
    def updateField(self, u1, u2):
        '''
        Update the field array given the connectivity matrix and the spin configurtiaon, can update given an 
        input or not.
        
        Parameters:
        u1 (flaot): input into the first configuration
        u2 (float): input into the second configuration 
        '''
        
        if self.input_u == True:

            
            self.field_matrix1 = np.add(self.J@self.spins1,self.encoder*u1)
            self.field_matrix2 = np.add(self.J@self.spins2,self.encoder*u2) 
            
        else: 
            self.field_matrix1 = self.J@self.spins1 
            self.field_matrix2 = self.J@self.spins2 
        
    
    def getField(self):
        return self.field_matrix1, self.field_matrix2
    
    def getProbabilitySpin(self, field_matrix):
        '''
        Define the probability that the spin when updated equals +1, 1-prob = spin equals -1.
        '''
        prob = (1/(1+np.exp(-2*field_matrix.astype(float)/self.temperature))) #probability of having a 1
        
        
        return prob
    
    def updateSpin(self, field_matrix):
        '''
        Update spins based on field and the probability of the spin value
        '''

        spin_new = np.ones(self.n_spins)
        prob = self.getProbabilitySpin(field_matrix)
        rand_vals = np.random.uniform(0,1, self.n_spins) #if value is below prob, set spin to +1, else to -1

      
        for i in range(self.n_spins): 
            if rand_vals[i] > prob[i]:
                spin_new[i] = -1
                

        return spin_new.astype(int), prob, rand_vals
    
    
        
    def simulateOverlap(self, epochs, verbose = False):
        '''
        Run one n spin updates and track the evolution of the magnetization, overlap and input.
        '''
        #compute the initial overlap between configurations, defined as the normalized dot product
        init_q = np.dot(self.spins1.T,self.spins2)/self.n_spins        
        q = [init_q]
        
        #define two different input sequences
        u1_array = np.random.binomial(1, self.r,epochs)*2-1 + self.mean_u
        u2_array = np.random.binomial(1, self.r,epochs)*2-1 + self.mean_u
        
        #initialize the initial magnetization of each of the configurations with the macroscopically aligned pattern
        m1_init = np.dot(self.p[0,:],self.spins1)/self.n_spins
        m2_init = np.dot(self.p[0,:],self.spins2)/self.n_spins
        
        #initialize the initial magnetization of each of the configurations with the second pattern
        m21_init = np.dot(self.p[1,:],self.spins1)/self.n_spins
        m22_init = np.dot(self.p[1,:],self.spins2)/self.n_spins
        
        #initialize the error in q due to N not being infinite
        s12_init = np.dot((m1_init*self.p[0,:] - self.spins1),
                             (m2_init*self.p[0,:]- self.spins2))/self.n_spins
        
        m1 = [m1_init]
        m2 = [m2_init]
        m21 = [m21_init]
        m22 = [m22_init]
        s12 = [s12_init]

        for e in range(epochs): 
            
            #update each of the spin configurations based on the current field matrix
            self.spins1, prob1, rand_v1 = self.updateSpin(self.field_matrix1)
            self.spins2, prob2, rand_v2 = self.updateSpin(self.field_matrix2)

            #update magnetizations and error
            m1_new = np.dot(self.p[0,:],self.spins1)/self.n_spins
            m2_new = np.dot(self.p[0,:],self.spins2)/self.n_spins
            s12_new = np.dot((m1_new*self.p[0,:] - self.spins1),
                             (m2_new*self.p[0,:]- self.spins2))/self.n_spins            
            m21_new = np.dot(self.p[1,:],self.spins1)/self.n_spins
            m22_new = np.dot(self.p[1,:],self.spins2)/self.n_spins
            #append magnetization and error
            m1.append(m1_new)
            m2.append(m2_new)
            s12.append(s12_new)            
            m21.append(m21_new)
            m22.append(m22_new)
            
            #update and append new overlap of the two configurations
            new_q = (self.spins1.T@self.spins2)/self.n_spins
            q.append(new_q)
            #give in a different input to each of the spin configurations
            if self.input_same == False:
                self.updateField(u1_array[e], u2_array[e])
            
            #give in the same input to the spin configuraitons
            if self.input_same == True:
                self.updateField(u1_array[e], u1_array[e])

            if verbose == True:
                print("Epoch " + str(e+1)+ "= " + str(new_mag) + " temp " + str(T))

        return q, prob1, prob2, rand_v1, rand_v2, self.spins1, self.spins2, m1, m2, s12, m21, m22, u1_array, u2_array

# 3. Evolution of alignment of two configurations

Visualize the evolution as a function of time of two different spin configuraitons given a set of temperature (T) and input rate (r) values

In [ ]:
n_t = 15
n_r = 11
n_spins = int(1e3)
p = 3
overlap = 0.8
q = 0.8
n_C= 3
mu = 0
epochs = 50

data = np.zeros((n_t,n_r,epochs))#empty data array

t_range = np.linspace(0.001, 10, n_t)
r_range = np.linspace(0,1,n_r)

IS_s = Ising_setup(n_spins = n_spins, p=p, overlap = overlap, q = q, n_C = n_C)
J_mat, C_mat, pattern_matrix, spins1, spins2, encoder = IS_s.setWeights()

for i, t in enumerate(t_range):
    for j,r in enumerate(r_range):
        IS_r = Ising_run(spins1, spins2, pattern_matrix, J_mat, encoder, r = r, mean_u = mu, temperature = t)
        q, p1, p2, r1, r2, s1, s2, m1, m2, si12, m21, m22, u1, u2 = IS_r.simulateOverlap(epochs)
        data[i,j,:] = q[0:-1]

In [ ]:
#plot for a fixed r

idx_r = 8

plt.figure(figsize=(10,5))
plt.plot(np.arange(epochs),data[:,idx_r,:].T)
plt.xlabel("Time")
plt.ylabel("Configuration overlap (q)")
plt.title("Configuration overlap for an input rate of r= " +str(r_range[idx_r]))
#plt.legend(["Letter condition", allLetters[i] for i in range(0,10)])
plt.legend(["Temperature {}".format(np.round(t/n_C,2)) for t in t_range], bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
#plot for a fixed t

idx_t = 13

plt.figure(figsize=(10,5))
plt.plot(np.arange(epochs),data[1,:,:].T)
plt.xlabel("Time")
plt.ylabel("Configuration overlap (q)")
plt.title("Configuration overlap for an input temperature= " +str(np.round(t_range[idx_t]/n_C,4)))
#plt.legend(["Letter condition", allLetters[i] for i in range(0,10)])
plt.legend(["Rate {}".format(r) for r in r_range], bbox_to_anchor=(1.05, 1), loc='upper left')

# 4. Theoretical evolution of the magnetization with the encoder

This section looks at the evolution of the overlap of the encoder with one configuration and the magnetization of each of the configurations along the macroscopically aligned pattern. Different initial values for encoder alignment with the macriscopic pattern are tested. The simulations are run for fixed values of temperature (T) and input rate (r).


In [ ]:
'Network parameters'
n_spins = int(2e4) #number of spins in the network
p = 2 #number of patterns to encode in the network
overlap = 0.8 #initial value of the overlap of each configuration with the macroscopic pattern
ov_q = 0.9 #alignment of the two patterns
n_C= 3 #network connectivity
T = 0.75 #temperature for the simulaiton
epochs = 200 #number of time points to run

'Input parameters'
mu = 0 #bias of the input
rate = 0.5 #rate of input being +/-1
n_e = 6 #number of different encoder overlap values to test
ov_encoder = np.linspace(0,1,n_e) #array of values of encoder and macroscopic pattern overlap



'Initialize arrays for the evolution of the simulations'
data = np.zeros((n_e,epochs))#data array for the overlap of two configurations 
#data arrays for the magnetization of each configuration with the macroscopic pattern
magnetization1 = np.zeros((n_e,epochs))
magnetization2 = np.zeros((n_e,epochs))

#Fill in the data arrays for the given networks defined and for a range of encoder overlaps
for i, e in enumerate(ov_encoder):
    IS_s = Ising_setup(n_spins = n_spins, p=p, overlap = overlap, overlap_encoder = e, q = ov_q, n_C = n_C)
    J_mat, C_mat, pattern_matrix, spins1, spins2, encoder = IS_s.setWeights()
    IS_r = Ising_run(spins1, spins2, pattern_matrix, J_mat, encoder, r = rate, mean_u = mu, temperature = T, input_u = True)
    q, p1, p2, r1, r2, s1, s2, m1, m2, si12, m21, m22, u1, u2 = IS_r.simulateOverlap(epochs)
    
    #overlap between configurations
    data[i,:] = q[0:-1]
    
    #magnetization of configurations with macroscopic pattern
    magnetization1[i,:] = m1[0:-1] 
    magnetization2[i,:] = m2[0:-1]
    

In [ ]:
#plot the overlap of the two configurations

plt.figure(figsize=(10,5))
plt.plot(np.arange(epochs),data[:,:].T)
plt.xlabel("Time")
plt.ylabel("Configuration overlap (q)")
plt.title("Configuration overlap for an input temperature= " +str(np.round(T/n_C,4)))
#plt.legend(["Letter condition", allLetters[i] for i in range(0,10)])
plt.legend(["Encoder overlap {}".format(e) for e in ov_encoder], bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
#plot magnetization of first configuration for different encoder overlaps

plt.figure(figsize=(10,5))
plt.plot(np.arange(epochs),magnetization1[:,:].T)
plt.xlabel("Time")
plt.ylabel("Magnetization for a single configuraiton")
plt.title("Magnetization for an input temperature= " +str(np.round(T/n_C,4)))
#plt.legend(["Letter condition", allLetters[i] for i in range(0,10)])
plt.legend(["Encoder overlap {}".format(e) for e in ov_encoder], bbox_to_anchor=(1.05, 1), loc='upper left')


# 5. Analytical update of the overlap of two configurations

Analaytically calculate the evolution of two configurations according to: B. Derrida, E. Gardner, and A. Zippelius. An exactly solvable asymmetric neural network model. EPL, 4(2), 1987.

In [ ]:
#Analytical update as in Derrida 1987

def analyticUpdate_m(m, a, T):
    '''
    Function takes the mean magnetization, a fixed temperature and capacity and returns the mean at the 
    next time step.
    '''
        
    g = lambda x: (np.exp(-x**2)*np.tanh((m-x*np.sqrt(2*a))/T))/np.sqrt(np.pi) #euqation (18) in Derrida 1987 paper
    f = integrate.quad(g, -np.inf, np.inf)

    return f[0]

def analyticUpdate_q(m1, m2, q, a, T, epochs):
    '''
    Function takes the mean magnetization for both configurations, the previous q value, a fixed temperature and 
    capacity and returns the q at the next time step. Note that this analytical approach assumes C (average graph
    connectivity) and p (number of stored patterns) go to infinity.
    
    Parameters:
    m1 (float): magnetization of the first configuraiton along the macroscopic pattern
    m2 (float): magnetization of the second configuration along the macroscopic pattern
    q (float): initial overlap of the two configurations
    a (float): capacity
    T (float): temperature
    '''
    
    f_q = [q]
    f_m1 = [m1]
    f_m2 = [m2]
   
    for e in range(epochs):
        
        #due to floating poitn error, f_q can sometimes be above 1, which is not possible and will lead to neg_factor
        #being undefined, thus threshold it to 1.
        if f_q[e] > 1:
            f_q[e] = 1
        pos_factor = np.sqrt(a*(1+f_q[e]))
        neg_factor = np.sqrt(a*(1-f_q[e]))
        #euqation (27) in Derrida 1987 paper
        g_q = lambda y, x: (np.exp(-y**2 -x**2)*np.tanh((f_m1[e]-y*pos_factor - x*neg_factor)/T)
                            *np.tanh((f_m1[e]-y*pos_factor -x*neg_factor)/T))/np.pi
        f = integrate.dblquad(g_q, -np.inf, np.inf, -np.inf, np.inf)
        f_q.append(f[0])
        
        #compute m1 and m2 analytically at the given time step
        m1_next = analyticUpdate_m(f_m1[e], a, T)
        m2_next = analyticUpdate_m(f_m2[e], a, T)
        
        f_m1.append(m1_next)
        f_m2.append(m2_next)
        
    return f_q, f_m1, f_m2

def analyticUpdate_q_same_m(m, q, a, T, epochs):
    '''
    Function takes the mean magnetization for both configurations (same value, the previous q value, a fixed 
    temperature and capacity and returns the q at the next time step. Note that this analytical approach 
    assumes C (average graph connectivity) and p (number of stored patterns) go to infinity.
    
    Parameters:
    m (float): magnetization of both configuraitons along the macroscopic pattern
    q (float): initial overlap of the two configurations
    a (float): capacity
    T (float): temperature
    '''
    
    f_q = [q]
    f_m = [m]
   
    for e in range(epochs):
               
        pos_factor = np.sqrt(a*(1+f_q[e]))
        neg_factor = np.sqrt(a*(1-f_q[e]))
        
        #euqation (27) in Derrida 1987 paper
        g_q = lambda y, x: (np.exp(-y**2 -x**2)*2*np.tanh((f_m[e]-y*pos_factor - x*neg_factor)/T))/np.pi
        f = integrate.dblquad(g_q, -np.inf, np.inf, -np.inf, np.inf)
        f_q.append(f[0])
        
        #compute m1 and m2 analytically at the given time step
        m_next = analyticUpdate_m(f_m[e], a, T)
                
        f_m.append(m_next)
        
    return f_q, f_m

### 5.1 Run analytical update for a series of time steps and initial encoder alignment values

In [ ]:
n_t = 10
n_a = 10
m_init = 0.6
q_init = 0.9

epochs = 15

data_q = np.zeros((n_t,n_a,epochs+1))
data_m1 = np.zeros((n_t,n_a,epochs+1))
data_m2 = np.zeros((n_t,n_a,epochs+1))

t_range = np.linspace(0.001, 2, n_t)
a_range = np.linspace(0.1,2,n_a)

for i, t in enumerate(t_range):
    for j,a in enumerate(a_range):
        f_q, f_m1, f_m2 = analyticUpdate_q(m1 = m_init, m2 = m_init, q = q_init, a = a, T = t, epochs = epochs)
        data_q[i,j,:] = f_q
        data_m1[i,j,:] = f_m1 
        data_m2[i,j,:] = f_m2

In [ ]:
#save the data
data_path = "/home/elosegui/MSc_thesis_project/numpy_results/derrida_model/analytical_q/"

np.save(data_path+"config_overlap_qinit" +str(q_init)+"_m_init"+str(m_init), data_q)
np.save(data_path+"magnetization_config1" +str(q_init)+"_m_init"+str(m_init), data_m1)
np.save(data_path+"magnetization_config2" +str(q_init)+"_m_init"+str(m_init), data_m2)

#load the data
data_q = np.load(data_path+"config_overlap_qinit" +str(q_init)+"_m_init"+str(m_init)+'.npy')

### 5.3 Plots for two network configuration overlap as a funciton of temperature and capacity

In [ ]:
#plot of configuration overlap for a range of temperature values given network capacity

idx_a = 5 #index of capacity to plot

plt.figure(figsize=(10,5))
plt.plot(np.arange(epochs+1),data_q[:,idx_a,:].T)
plt.xlabel("Time")
plt.ylabel("Configuration overlap (q)")
plt.title("Configuration overlap for a network capacity of a = " +str(a_range[idx_a]))
plt.legend(["Temperature {}".format(np.round(t,3)) for t in t_range], bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
#plot configuration overlap for a range of capacity values and a given temperature

idx_t = 2 #index of temperature

plt.figure(figsize=(10,5))
plt.plot(np.arange(epochs+1),data_q[idx_t,:,:].T)
plt.xlabel("Time")
plt.ylabel("Configuration overlap (q)")
plt.title("Configuration overlap for a network temperature of t = " +str(t_range[idx_t]))
#plt.legend(["Letter condition", allLetters[i] for i in range(0,10)])
plt.legend(["Capacity {}".format(np.round(a,2)) for a in a_range], bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
#plot overlap of two configurations for a range of temperature and capacity values

path_plots = "/home/elosegui/MSc_thesis_project/numpy_results/derrida_model/analytical_q/"

x1, y1 = np.meshgrid(a_range, t_range)
plt.contourf(x1,y1,data_q[:,:,-1])
plt.xlabel("capacity (alpha)")
plt.ylabel("Temperature (T)")
plt.title("Convergence of configuration overlap")
plt.colorbar()

plt.savefig(path_plots+"config_overlap_qinit" +str(q_init)+"_m_init"+str(m_init), format='png')